In [103]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.utils import to_categorical
import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [104]:
col = ['feature2','feature3','feature4','class']


In [105]:
train_data = pd.read_csv("train_dec08_task4_missing.csv")
test_data = pd.read_csv("test_dec08_task4_missing_only_features.csv")

In [106]:
train_data = train_data[col].copy()
train_data = train_data.dropna()

In [107]:
train_data.iloc[1:10]

,feature2,feature3,feature4,class
1,20.176954,15.535818,15.857875,A
2,19.973465,15.704548,2.553840,A
3,7.418786,7.192065,10.894859,A
4,20.001505,17.789371,0.435783,A
5,10.191896,16.274527,13.826648,A
6,23.788850,16.358599,1.958868,A
8,28.216307,19.733914,14.847692,A
11,23.210151,24.682622,-7.276298,A
12,17.421660,17.454866,18.429444,A


In [108]:
train_data['class'].value_counts()

C    2111
B    1668
A     842
D     822
Name: class, dtype: int64

In [109]:
x = pd.DataFrame(data = None, columns= train_data.columns)

In [110]:
from sklearn.utils import resample
classifications = ['C','B']
np1 = np.array([])
for c in classifications:
  np1 = np.append(np1,(resample(train_data[train_data['class'] == c],
             replace=False,
             n_samples=800,
             random_state=42)))
np1 = np.append(np1,np.array(train_data[train_data['class'] == 'A']))
np1 = np.append(np1,np.array(train_data[train_data['class'] == 'D']))
x = pd.DataFrame(data = np1.reshape(int(13056/4),4), columns= train_data.columns)
#x.append([train_data[train_data['class'] == 'A'],train_data[train_data['class'] == 'D']])


In [111]:
x

,feature2,feature3,feature4,class
0,18.708037,16.125312,18.669622,C
1,9.039289,25.206686,6.112024,C
2,11.949262,26.097404,6.470531,C
3,7.080678,31.390979,33.015794,C
4,12.169515,25.032152,4.018233,C
...,...,...,...,...
3259,9.814982,13.292516,5.816327,D
3260,27.959554,11.679617,-0.046006,D
3261,21.556543,9.799886,2.398715,D
3262,11.728741,11.899123,8.347815,D


In [112]:
x['class'].value_counts()

A    842
D    822
C    800
B    800
Name: class, dtype: int64

In [113]:
X = x[['feature2','feature3','feature4']]
y = x['class']

In [114]:
from sklearn import preprocessing
zscore = preprocessing.StandardScaler()
X = zscore.fit_transform(X)
t = zscore.fit_transform(test_data.iloc[:,2:5])

In [115]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, random_state=42)

In [116]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=20, min_samples_split=200)
clf.fit(X_train, y_train)

RandomForestClassifier(min_samples_split=200, n_estimators=20)

In [117]:
from sklearn.model_selection import cross_validate
scores = cross_validate(clf,X_val,y_val,cv=5,scoring='accuracy')
scores

{'fit_time': array([0.03415442, 0.02924204, 0.02898526, 0.02701187, 0.02684522]),
 'score_time': array([0.0028832 , 0.00285769, 0.0041573 , 0.00278997, 0.00311947]),
 'test_score': array([0.32653061, 0.42857143, 0.40816327, 0.39795918, 0.3877551 ])}

# Deep Learning

In [118]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD

In [119]:
def toCategory(lst):
  t = []
  for i in range(0,len(lst)):
      k = lst
      if(k[i] == 'A'):
          t.append(0)
      elif(k[i] == 'B'):
          t.append(1)
      elif(k[i] == 'C'):
         t.append(2)
      elif(k[i] == 'D'):
         t.append(3)
  t = to_categorical(t, 4)
  return t;
Y = toCategory(y)

In [120]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.15, random_state = 42)

In [121]:
model = Sequential()
model.add(Dense(100, input_dim=3, activation='relu'))
model.add(Dense(35, activation='relu'))
model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics=['accuracy'])

In [122]:
model.fit(X_train, y_train, batch_size=33, epochs=20, validation_data=(X_test, y_test))

Epoch 1/20
85/85 [==============================] - 1s 5ms/step - loss: 1.2871 - accuracy: 0.3381 - val_loss: 1.2344 - val_accuracy: 0.4041
Epoch 2/20
85/85 [==============================] - 0s 2ms/step - loss: 1.2371 - accuracy: 0.3717 - val_loss: 1.2378 - val_accuracy: 0.3755
Epoch 3/20
85/85 [==============================] - 0s 3ms/step - loss: 1.2291 - accuracy: 0.3713 - val_loss: 1.2366 - val_accuracy: 0.3633
Epoch 4/20
85/85 [==============================] - 0s 3ms/step - loss: 1.2251 - accuracy: 0.3836 - val_loss: 1.2269 - val_accuracy: 0.3796
Epoch 5/20
85/85 [==============================] - 0s 3ms/step - loss: 1.2207 - accuracy: 0.3911 - val_loss: 1.2198 - val_accuracy: 0.3816
Epoch 6/20
85/85 [==============================] - 0s 3ms/step - loss: 1.2217 - accuracy: 0.3868 - val_loss: 1.2262 - val_accuracy: 0.3612
Epoch 7/20
85/85 [==============================] - 0s 3ms/step - loss: 1.2210 - accuracy: 0.3774 - val_loss: 1.2290 - val_accuracy: 0.3673
Epoch 8/20
85/85 [==

In [123]:
#answer = np.argmax(model.predict(test_data),axis = -1)

In [133]:
test_data = pd.DataFrame(t, columns = col[:-1]).fillna(0)
answer =clf.predict(test_data)

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [134]:
answer

array(['D', 'D', 'A', ..., 'C', 'A', 'D'], dtype=object)

In [126]:
#ans = []
#for a in answer:
 # if(a == 0):
 #   ans.append("A")
 # elif(a == 1):
 #   ans.append("B")
 # elif(a == 2):
 #   ans.append("C")
 # elif(a == 3):
 #   ans.append("D")

In [135]:
id = np.linspace(1,1600,1600,dtype = "int")
a = np.concatenate((id,answer))
a = a.reshape(2,1600)
a = np.transpose(a, axes=None)
df = pd.DataFrame(a, columns = ['Id','Category'])
df.to_csv('./answer4.csv', columns=['Id','Category'])